# Final Project Analysis Example using Thicket

In [4]:
import sys
sys.path.append("/scratch/group/csce435-f23/python-3.8.17/lib/python3.8/site-packages")
sys.path.append("/scratch/group/csce435-f23/thicket")
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd

import thicket as th

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)


In [11]:
df[~df.index.duplicated()]
tk = th.Thicket.from_caliperreader(glob("*.cali"))
tk.dataframe = tk.dataframe.drop(["nid", "spot.channel"], axis=1)
tk.dataframe

ValueError: cannot handle a non-unique multi-index!

In [ ]:
tk.metadata

In [ ]:
tk.statsframe.dataframe["time"] = 1
print(tk.tree())

In [ ]:
tk.metadata.columns

In [ ]:
gb = tk.groupby("InputSize")

In [ ]:
ctk = th.Thicket.concat_thickets(
    thickets=list(gb.values()),
    headers=list(gb.keys()),
    axis="columns",
    metadata_key="num_procs"
)

In [ ]:
ctk.dataframe

In [ ]:
# Drop unwanted columns
ctk.dataframe = ctk.dataframe.reset_index().drop(("node"), axis=1)
# Rename index to remove tuple
ctk.dataframe = ctk.dataframe.rename({("name", ""): "name", ("num_procs", ""): "num_procs"}).set_index(["name", "num_procs"])
# Drop "time" label as it is redundant
#ctk.dataframe.columns = ctk.dataframe.columns.droplevel(1)

In [ ]:
# Define common variables
function_names = set(ctk.dataframe.index.get_level_values("name"))
num_procs = sorted(list(set(ctk.dataframe.index.get_level_values("num_procs"))))
input_sizes = sorted(list(set(ctk.dataframe.columns.get_level_values(0))))

In [ ]:
def plot_mpl(df, title, xlabel, ylabel):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(
        df,
        marker='o',
    )
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend(df.columns)
    plt.grid(True)
   # plt.yscale("log", base=10)    plt.xticks(num_procs)
    plt.show()
    return fig, ax

In [ ]:
for name in function_names:
     for size in input_sizes:
        plot_mpl(
            df=ctk.dataframe.loc[name,size],
            title=f"Merge MPI, {name}, {size}", 
            xlabel="num_procs",
            ylabel="Time (seconds)",
        )